In [38]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats 

# Библиотека для хеширования
import hashlib

In [39]:
df = pd.read_csv('users.csv', index_col=0)
df.head()

,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


In [40]:
def hash_user_id(uid):
    """
    Функция на вход получает столбец user_id и возвращает user_hash
    Используется библиотека hashlib
    """
    uid_str = str(uid)
    uid_hash = hashlib.md5(uid_str.encode()).hexdigest()  # вернёт hex-строку
    return int(uid_hash, 16)  # переводим hex в int

In [41]:
df['user_hash'] = df['user_id'].apply(hash_user_id)
df['group_index'] = df['user_hash'] % 100 # Определяем остаток от деления, для разбивения на группы

df.head()

,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm,user_hash,group_index
0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low,207021514915257020430936527496092606591,91
1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high,275782213665482290192042867242558471677,77
2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high,103266185129541807356797093470683387217,17
3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low,3734281640716273381446497663233150109,9
4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low,11112181322636506944205271446330541724,24


In [42]:
def assign_group(index):
    return "A" if index < 50 else "B"

In [43]:
df["group"] = df["group_index"].apply(assign_group)
print(df["group"].value_counts(normalize=True))

group
B    0.50202
A    0.49798
Name: proportion, dtype: float64


In [44]:
group_counts = df["group"].value_counts()
print(group_counts)

group
B    50955
A    50545
Name: count, dtype: int64


In [45]:
A_count = group_counts.get("A", 0)
B_count = group_counts.get("B", 0)

absolute_diff = abs(A_count - B_count)
relative_diff = absolute_diff / ((A_count + B_count) / 2) * 100

print(f"Абсолютное отклонение: {absolute_diff}")
print(f"Относительное отклонение: {relative_diff:.2f}%")

Абсолютное отклонение: 410
Относительное отклонение: 0.81%


In [46]:
if relative_diff <= 5:
    print("Идеально: разница между группами не превышает 5%. Рандомизация качественная.")
elif relative_diff <= 10:
    print("Допустимо: разница до 10%. Можно использовать, но стоит быть внимательным.")
else:
    print("Критично: разница превышает 10%. Рандомизация требует доработки.")

Идеально: разница между группами не превышает 5%. Рандомизация качественная.


In [47]:
print('\nСравнение возраста:')
print(df.groupby('group')['age'].agg(['mean', 'median', 'std']))


Сравнение возраста:
            mean  median       std
group                             
A      25.937798    24.0  7.865046
B      25.875223    23.0  7.811688
